In [1]:
import pandas as pd
import numpy as np
import pyodbc
import geopandas as gpd
import sqlalchemy
from shapely import wkt
import h5py
import pandana as pdna

In [16]:
def read_from_sde(connection_string, feature_class_name, version,
                  crs={'init': 'epsg:2285'}, is_table = False):
    """
    Returns the specified feature class as a geodataframe from ElmerGeo.
    
    Parameters
    ----------
    connection_string : SQL connection string that is read by geopandas 
                        read_sql function
    
    feature_class_name: the name of the featureclass in PSRC's ElmerGeo 
                        Geodatabase
    
    cs: cordinate system
    """


    engine = sqlalchemy.create_engine(connection_string)
    con=engine.connect()
    #con.execute("sde.set_current_version {0}".format(version))
    if is_table:
        gdf=pd.read_sql('select * from %s' % 
                   (feature_class_name), con=con)
        con.close()

    else:
        df=pd.read_sql('select *, Shape.STAsText() as geometry from %s' % 
                   (feature_class_name), con=con)
        con.close()

        df['geometry'] = df['geometry'].apply(wkt.loads)
        gdf=gpd.GeoDataFrame(df, geometry='geometry')
        gdf.crs = crs
        cols = [col for col in gdf.columns if col not in 
                ['Shape', 'GDB_GEOMATTR_DATA', 'SDE_STATE_ID']]
        gdf = gdf[cols]
    
    return gdf

In [17]:
input_dir = r'C:\Workspace\VisionEval\models\VERSPM\inputs_RVMPO'
output_dir = r'C:\Workspace\VisionEval\models\VERSPM\inputs'

regional_geo = 'PSRC'

In [19]:
# Load data that will be used for multiple fields
connection_string = 'mssql+pyodbc://AWS-PROD-SQL\Sockeye/ElmerGeo?driver=SQL Server?Trusted_Connection=yes'
crs = {'init' : 'EPSG:2285'}
version = "'DBO.Default'"
gdf_shp = read_from_sde(connection_string, 'blockgrp2020', version, crs=crs, is_table=False)
gdf_bg = read_from_sde(connection_string, 'blockgrp2020', version, crs=crs, is_table=False)

run_dir_18 = r'C:\Workspace\sc_2018_rtp_final\soundcast'
run_dir_50 = r'L:\RTP_2022\final_runs\sc_rtp_2050_constrained_final\soundcast'

parcel_18 = pd.read_csv(os.path.join(run_dir_18,'inputs\scenario\landuse\parcels_urbansim.txt'), delim_whitespace=True)
parcel_50 = pd.read_csv(os.path.join(run_dir_50,'inputs\scenario\landuse\parcels_urbansim.txt'), delim_whitespace=True)

C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [20]:
# Create lookup for parcels to block groups
# Load parcel centroids as geodataframe
parcel_18_gdf = gpd.GeoDataFrame(
    parcel_18, geometry=gpd.points_from_xy(parcel_18.XCOORD_P, parcel_18.YCOORD_P))
crs = {'init' : 'EPSG:2285'}
parcel_18_gdf.crs = crs

parcel_50_gdf = gpd.GeoDataFrame(
    parcel_50, geometry=gpd.points_from_xy(parcel_50.XCOORD_P, parcel_50.YCOORD_P))
crs = {'init' : 'EPSG:2285'}
parcel_50_gdf.crs = crs

parcel_18_bg_lookup = gpd.sjoin(gdf_bg, parcel_18_gdf)[['PARCELID','geoid20']]
parcel_50_bg_lookup = gpd.sjoin(gdf_bg, parcel_50_gdf)[['PARCELID','geoid20']]

parcel_18 = parcel_18.merge(parcel_18_bg_lookup, on='PARCELID')
parcel_50 = parcel_50.merge(parcel_50_bg_lookup, on='PARCELID')

# # Add the census tract back to the gdf
# parcel_18_gdf = parcel_18_gdf.merge(parcel_18_bg_lookup, on='PARCELID')
# parcel_50_gdf = parcel_50_gdf.merge(parcel_50_bg_lookup, on='PARCELID')

def fill_dummy_parcels(parcel_df, gdf_shp):
    # Add some dummy variables for block groups that do not have any parcels 
    diff_list = list(set(gdf_shp['geoid20'].unique()) - set(parcel_df['geoid20'].unique()))

    # Add empty rows that represent this geoid
    temp_df = parcel_df.iloc[0:len(diff_list)].copy()

    temp_df[temp_df.columns] = 0
    temp_df['geoid20'] = diff_list
    parcel_df = parcel_df.append(temp_df)

    return parcel_df

# parcel_18_gdf = fill_dummy_parcels(parcel_18_gdf, gdf_shp)
# parcel_50_gdf = fill_dummy_parcels(parcel_50_gdf, gdf_shp)



parcel_18 = fill_dummy_parcels(parcel_18, gdf_shp)
parcel_50 = fill_dummy_parcels(parcel_50, gdf_shp)

# Give dummy parcels an XY coordinate at centroid of their block groups
block_group_list = parcel_18[parcel_18['PARCELID'] == 0]['geoid20']

parcel_18.loc[parcel_18['PARCELID'] == 0, 'XCOORD_P'] = gdf_shp[gdf_shp['geoid20'].isin(block_group_list)].centroid.x.values
parcel_18.loc[parcel_18['PARCELID'] == 0, 'YCOORD_P'] = gdf_shp[gdf_shp['geoid20'].isin(block_group_list)].centroid.y.values
# parcel_18[parcel_18['PARCELID'] == 0]


# # Give dummy parcels an XY coordinate at centroid of their block groups
block_group_list = parcel_50[parcel_50['PARCELID'] == 0]['geoid20']

parcel_50.loc[parcel_50['PARCELID'] == 0, 'XCOORD_P'] = gdf_shp[gdf_shp['geoid20'].isin(block_group_list)].centroid.x.values
parcel_50.loc[parcel_50['PARCELID'] == 0, 'YCOORD_P'] = gdf_shp[gdf_shp['geoid20'].isin(block_group_list)].centroid.y.values


# # Recreate the gdf

parcel_18_gdf = gpd.GeoDataFrame(
    parcel_18, geometry=gpd.points_from_xy(parcel_18.XCOORD_P, parcel_18.YCOORD_P))
crs = {'init' : 'EPSG:2285'}
parcel_18_gdf.crs = crs

parcel_50_gdf = gpd.GeoDataFrame(
    parcel_50, geometry=gpd.points_from_xy(parcel_50.XCOORD_P, parcel_50.YCOORD_P))
crs = {'init' : 'EPSG:2285'}
parcel_50_gdf.crs = crs

# parcel_18_gdf = parcel_18_gdf.merge(parcel_18_bg_lookup, on='PARCELID')
# parcel_50_gdf = parcel_50_gdf.merge(parcel_50_bg_lookup, on='PARCELID')

C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When m

In [6]:
# parcel_18_gdf[parcel_18_gdf['geoid20'] == '530330228033']

In [7]:
# Write an initial Geo file
# Note that we only want to include zones that have households located in them
fname = 'geo.csv'
df = pd.read_csv(os.path.join(input_dir,'..\defs',fname))
df.head()

,Azone,Bzone,Czone,Marea
0,PSRC,530330001011,NaN,PSRC
1,PSRC,530330001012,NaN,PSRC
2,PSRC,530330001013,NaN,PSRC
3,PSRC,530330001021,NaN,PSRC
4,PSRC,530330001022,NaN,PSRC


In [8]:
# Define geogrpahy
# For now make an exact replica fo the RVMPO data with Azone = region, Bzone = block groups
# Ideally we have Azone = county, Bzone = block groups

# Get Census tracts from the region
# Load  tract geographies from ElmerGeo
# NOTE: We are using 2020 geographies;
# Use geoid20 as the field name


# Load cities shapefile
# gdf_cities = read_from_sde(connection_string, 'cities', version, crs=crs, is_table=False)
len(df)


2928

In [9]:
# gdf_seattle = gdf_cities[gdf_cities['city_name'] == 'Seattle']
# geoid_list = gpd.sjoin(gdf_shp, gdf_seattle)['geoid20']

# # 
# df_geog = gdf_shp[['geoid20']]
# geoid_list = geoid_list[geoid_list != '530330053041']
# ## For now let's just select a sub sample of Block groups in Seattle
# df_geog = df_geog[df_geog['geoid20'].isin(geoid_list)]

In [10]:

# geoid_list = ['530330028004','530330028001','530330028002','530330028003']
# df_geog = gdf_shp[['geoid20']]
# df_geog = df_geog[df_geog['geoid20'].isin(geoid_list)]


In [11]:
df_geog = gdf_bg[['geoid20']]
df_geog.rename(columns={'geoid20': 'Bzone'}, inplace=True)
df_geog.loc[:,'Azone'] = 'PSRC'
df_geog.loc[:,'Marea'] = 'PSRC'
# df_geog.loc[:,'Czone'] = range(len(df_geog))
df_geog.loc[:,'Czone'] = "NA"
df_geog[df.columns].head()

C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

,Azone,Bzone,Czone,Marea
0,PSRC,530330001011,NA,PSRC
1,PSRC,530330001012,NA,PSRC
2,PSRC,530330001013,NA,PSRC
3,PSRC,530330001021,NA,PSRC
4,PSRC,530330001022,NA,PSRC


In [16]:

df_geog[df.columns].to_csv(os.path.join(output_dir,r'../defs',fname), index=False)

In [17]:
# Get list of parcels in the block group area

# geoid_list = ['530330028004','530330028001','530330028002','530330028003']
# # df_geog = gdf_shp[['geoid20']]
# temp_df_geog = gdf_shp[gdf_shp['geoid20'].isin(geoid_list)]
# len(df_geog)
# parcel_list = gpd.sjoin(temp_df_geog, parcel_18_gdf)['PARCELID']

In [18]:
# # input_dir = r'C:\Workspace\VisionEval\models\VERSPM\inputs'
# # output_dir = r'C:\Workspace\VisionEval\input_creation\psrc_inputs'

# fname = 'azone_carsvc_characteristics.csv'
# df = pd.read_csv(os.path.join(input_dir,fname))
# df.head()

# Update A-Zone files
Currently reprsenting as for full region
According to docs, should be about the size of a PUMA

In [19]:
fname = 'azone_charging_availability.csv'
df = pd.read_csv(os.path.join(input_dir,fname))
df['Year'] = ['2018','2050']

In [20]:
# Many of these we are not yet updating with region-specific numbers and are using the defaults
# However, we need to change the region name in the column
# additionally, we need to provide specific years of data
# Using 2018 and 2050 for years now; we could specify all model years 

def update_azone(df, name):
    df['Geo'] = name
    df['Year'] = ['2018','2050']
    return df

def process_azone(fname):
    df = pd.read_csv(os.path.join(input_dir,fname))
    df = update_azone(df, 'PSRC')
    df.to_csv(os.path.join(output_dir,fname), index=False)


# This file specifies the different characteristics for high and low car service level and is used in the 
# CreateVehicleTable and AssignVehicleAge modules.
process_azone(fname)

fname = 'azone_charging_availability.csv'
# This file has data on proportion of different household types who has EV charging available and is used in the 
# AssignHHVehiclePowertrain module.
process_azone(fname)

fname = 'azone_electricity_carbon_intensity.csv'
# This file is used to specify the carbon intensity of electricity and is optional (only needed if user wants to modify the values). 
# The file is used in Initialize (VEPowertrainsAndFuels) and CalculateCarbonIntensity modules.
process_azone(fname)


fname = 'azone_fuel_power_cost.csv'
# This file supplies data for retail cost of fuel and electricity and is used in the CalculateVehicleOperatingCost module.
process_azone(fname)

fname = 'azone_hh_veh_mean_age.csv' 
# This file provides inputs for mean auto age and mean light truck age and is used in the AssignVehicleAge module.
process_azone(fname)

fname = 'azone_hh_veh_own_taxes.csv' 
# This file provides inputs for flat fees/taxes (i.e. annual cost per vehicle) and ad valorem taxes (i.e. percentage of vehicle value paid in taxes). The file is used in CalculateVehicleOwnCost module.
process_azone(fname)

fname = 'azone_payd_insurance_prop.csv' 
# This file provides inputs on the proportion of households having PAYD (pay-as-you-drive) insurance and is used in the CalculateVehicleOwnCost module.
process_azone(fname)

fname = 'azone_prop_sov_dvmt_diverted.csv' 
# This file provides inputs for a goal for diverting a portion of SOV travel within a 20-mile tour distance and is used in the DivertSovTravel module.
process_azone(fname)

# fname = 'azone_relative_employment.csv' 
# # This file contains ratio of workers to persons by age and is used in the PredictWorkers module.
# process_azone(fname)

fname = 'azone_veh_use_taxes.csv' 
# This file supplies data for vehicle related taxes and is used in the CalculateVehicleOperatingCost module.
process_azone(fname)

fname = 'azone_vehicle_access_times.csv' 
# This file supplies data for vehicle acces`s and egress time and is used in the CalculateVehicleOperatingCost module.
process_azone(fname)

In [21]:
fname = 'azone_gq_pop_by_age.csv' 
df = pd.read_csv(os.path.join(input_dir,fname))
# This file contains group quarters population estimates/forecasts by age and is used in the CreateHouseholds module.

# Get group quarters from parcel file

# Taking group quarters populations to be Zero in Soundcast
df[['GrpAge0to14','GrpAge15to19','GrpAge20to29','GrpAge30to54','GrpAge55to64','GrpAge65Plus']] = 0 
df['Geo'] = regional_geo
df['Year'] = ['2018','2050']
df.to_csv(os.path.join(output_dir,fname), index=False)
# parcel_18.columns

In [22]:
fname = 'azone_hh_pop_by_age.csv' 
# This file contains population estimates/forecasts by age and is used in the CreateHouseholds module.
df = pd.read_csv(os.path.join(input_dir,fname))

# Get this data from synthetic household/persons


def get_age_population(syn_h5, year):

    df_person = pd.DataFrame()
    for col in syn_h5['Person'].keys():
        df_person[col] = syn_h5['Person'][col][:]

    # df_hh = pd.DataFrame()
    # for col in ['hhno','hhparcel']:
    #     df_hh[col] = syn_h5['Household'][col][:]

    # df_person = df_person.merge(df_hh, how='left', on='hhno')
    # Filter for people living in Seattle only
    # df_person = df_person[df_person['hhparcel'].isin(parcel_list)]
    col_list = [i for i in df.columns if i not in ['Geo','Year']]

    # Separate ages into groups
    bins = [-1,14,19,29,54,64,200]
    df_person['new_group'] = pd.cut(df_person['pagey'], bins, labels=col_list)

    _df = df_person.groupby('new_group').count()[['psexpfac']].reset_index().T
    _df = _df.reset_index(drop=True)
    _df.columns = _df.iloc[0]
    _df = _df.iloc[1:]
    # _df.drop('new_group', axis=1, inplace=True)
    _df['Year'] = year
    _df['Geo'] = 'PSRC'
    _df = _df.reset_index(drop=True)
    
    return _df

syn_h5 = h5py.File(r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2018\new_emp\hh_and_persons.h5', 'r')
_df_18 = get_age_population(syn_h5, '2018')
_df_18 = _df_18[df.columns]
syn_h5.close()

syn_h5 = h5py.File(r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2050\rtp_2050\hh_and_persons.h5', 'r')
_df_50 = get_age_population(syn_h5, '2050')
_df_50 = _df_50[df.columns]
syn_h5.close()

_df = _df_18.append(_df_50)
_df.to_csv(os.path.join(output_dir,fname), index=False)




In [138]:
fname = 'azone_hhsize_targets.csv' 
# This file contains the household specific targets and is used in CreateHouseholds module.
df = pd.read_csv(os.path.join(input_dir,fname))


def get_hhsize_targets(syn_h5):
    df_hh = pd.DataFrame()
    for col in syn_h5['Household'].keys():
        df_hh[col] = syn_h5['Household'][col][:]
    mean_hh_size = df_hh['hhsize'].mean()
    one_person_hh = len(df_hh[df_hh['hhsize'] == 1])/len(df_hh)
    
    return mean_hh_size, one_person_hh

df['Year'] = ['2018','2050']
df['Geo'] = 'PSRC'

syn_h5 = h5py.File(r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2018\new_emp\hh_and_persons.h5', 'r')
mean_hh_size, one_person_hh = get_hhsize_targets(syn_h5)
df.loc[df['Year'] == '2018', 'AveHhSize'] =  mean_hh_size
df.loc[df['Year'] == '2018', 'Prop1PerHh'] =  one_person_hh

syn_h5 = h5py.File(r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2050\rtp_2050\hh_and_persons.h5', 'r')
mean_hh_size, one_person_hh = get_hhsize_targets(syn_h5)
df.loc[df['Year'] == '2050', 'AveHhSize'] =  mean_hh_size
df.loc[df['Year'] == '2050', 'Prop1PerHh'] =  one_person_hh

df.to_csv(os.path.join(output_dir,fname), index=False)

In [148]:
df_person = pd.DataFrame()
syn_h5 = h5py.File(r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2018\new_emp\hh_and_persons.h5', 'r')
for col in syn_h5['Person'].keys():
    df_person[col] = syn_h5['Person'][col][:]
# mean_hh_size = df_hh['hhsize'].mean()
# one_person_hh = len(df_hh[df_hh['hhsize'] == 1])/len(df_hh)


In [153]:
df_hh['hhincome'].sum()/len(df_person)

42405.91717289795

In [25]:
fname = 'azone_lttrk_prop.csv' 
# This file specifies the light truck proportion of the vehicle fleet and is used in AssignVehicleType module.
# This refers I believe to the main vehicle population; 
# Not changing the default for now, but should be looked up via MOVES or assumed flat?
df = pd.read_csv(os.path.join(input_dir,fname))
df['Geo'] = 'PSRC'
df['Year'] = ['2018','2050']
df.to_csv(os.path.join(output_dir,fname), index=False)

In [21]:

fname = 'azone_per_cap_inc.csv' 
# This file contains information on regional average per capita household and group quarters income in year 2010 dollars and is used in the PredictIncome module.
df = pd.read_csv(os.path.join(input_dir,fname))
df['Geo'] = 'PSRC'
df['Year'] = ['2018','2050']

# Get average income from h5 files
syn_h5 = h5py.File(r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2018\new_emp\hh_and_persons.h5', 'r')
df_hh = pd.DataFrame()
for col in syn_h5['Household'].keys():
    df_hh[col] = syn_h5['Household'][col][:]
# df_hh = df_hh[df_hh['hhparcel'].isin(parcel_list)]    
df_hh['per_cap_income'] = df_hh['hhincome']/df_hh['hhsize']
df.loc[df['Year'] == '2018', ['HHIncomePC.2010','GQIncomePC.2010']] = df_hh['per_cap_income'].mean()

syn_h5 = h5py.File(r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2050\rtp_2050\hh_and_persons.h5', 'r')
df_hh = pd.DataFrame()
for col in syn_h5['Household'].keys():
    df_hh[col] = syn_h5['Household'][col][:]
# df_hh = df_hh[df_hh['hhparcel'].isin(parcel_list)] 
df_hh['per_cap_income'] = df_hh['hhincome']/df_hh['hhsize']
df.loc[df['Year'] == '2050', ['HHIncomePC.2010','GQIncomePC.2010']] = df_hh['per_cap_income'].mean()

df.rename(columns={'HHIncomePC.2010': 'HHIncomePC.2018', 'GQIncomePC.2010': 'GQIncomePC.2018'}, inplace=True)

df.to_csv(os.path.join(output_dir,fname), index=False)

In [4]:
# import pandas as pd
# Get average vehicles per driver
#### FIXME: 
### us this to update the file directly
df = pd.read_csv(r'L:\RTP_2022\final_runs\sc_2018_rtp_final\soundcast\outputs\daysim\_household.tsv', delim_whitespace=True)

In [3]:
df['hhvehs'].sum()

3069100

In [12]:
# Get total number of adults
df['hhsize'].sum()-df['hhcu5'].sum()-df['hh515'].sum()

3314941

In [13]:
3069100/3314941.0


0.925838499086409

# B Zones
Block Group Level

In [28]:
fname = 'bzone_transit_service.csv' 
df = pd.read_csv(os.path.join(input_dir,fname))

# D4c
# Aggregate frequency of transit service within 0.25 miles of block group boundary per hour during evening peak period 
# (Ref: EPA 2010 Smart Location Database) from GTFS.
# See pg 23 https://www.epa.gov/sites/default/files/2021-06/documents/epa_sld_3.0_technicaldocumentationuserguide_may2021.pdf
# Aggregate Frequency of Peak Hour Transit Service (D4c) 

# FIXME: do a comparison to our calculation to make sure it lines up with the EPA data for base year


In [29]:
def calculate_transit_service(run_dir, year):
    # Transit stops do not have frequencies so we need to load the transit lines
    df_transit = gpd.read_file(os.path.join(run_dir,r'inputs\scenario\networks\shapefiles\AM\AM_edges.shp'))

    df = pd.read_csv(os.path.join(run_dir,r'inputs\scenario\networks\shapefiles\AM\AM_transit_segments.csv'))
    df_headways = pd.read_csv(os.path.join(run_dir,r'inputs\scenario\networks\headways.csv'))
    df = df.merge(df_headways[['LineID','hdw_16to17']])

    df_transit = df_transit.merge(df[['LineID','hdw_16to17','ij']], left_on='id', right_on='ij')

    gdf_bg = read_from_sde(connection_string, 'blockgrp2020', version, crs=crs, is_table=False)
    # Apply a quarter mile buffer around each block group
    gdf_bg.geography = gdf_bg.buffer(5280/4.0)

    gdf_joined = gpd.sjoin(gdf_bg, df_transit, how='left')

    bg_headways = gdf_joined.groupby(['geoid20','LineID']).mean()[['hdw_16to17']].reset_index()

    bg_headways['D4c'] = 60.0/bg_headways['hdw_16to17']
    bg_headways['D4c'].replace(np.inf, 0, inplace=True)

    bg_headways = bg_headways.groupby('geoid20').sum()[['D4c']]
    bg_headways = bg_headways.reset_index()

    # Make sure to include all block groups even if they have no transit
    add_df = gdf_bg[-gdf_bg['geoid20'].isin(bg_headways['geoid20'])][['geoid20']]
    add_df['D4c'] = 0
    bg_headways = bg_headways.append(add_df)

    bg_headways.rename(columns={'geoid20': 'Geo'}, inplace=True)
    bg_headways['Year'] = year

    # bg_headways = bg_headways[bg_headways['Geo'].astype('str').isin(geoid_list)]

    geo_df = pd.read_csv(os.path.join(output_dir,r'..\defs\geo.csv'))
    bg_headways = bg_headways[bg_headways['Geo'].isin(geo_df['Bzone'].astype('str'))]

    return bg_headways



In [30]:
run_dir = r'C:\Workspace\sc_2018_rtp_final\soundcast'
df = calculate_transit_service(run_dir, '2018')

C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\geopandas\geodataframe.py:182: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  super(GeoDataFrame, self).__setattr__(attr, val)


In [31]:
run_dir = r'L:\RTP_2022\final_runs\sc_rtp_2050_constrained_final\soundcast'
df_50 = calculate_transit_service(run_dir, '2050')

C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\geopandas\geodataframe.py:182: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  super(GeoDataFrame, self).__setattr__(attr, val)


In [32]:
df = df.append(df_50)
df.to_csv(os.path.join(output_dir,fname), index=False)

In [33]:
df

,Geo,D4c,Year
0,530330001011,11.0,2018
1,530330001012,23.0,2018
2,530330001013,23.0,2018
3,530330001021,12.0,2018
4,530330001022,35.0,2018
...,...,...,...
2915,530610538032,0.0,2050
2917,530610538034,0.0,2050
2924,530619400022,0.0,2050
2925,530619400023,0.0,2050


In [34]:
# gdf_bg[gdf_bg['geoid20'] == '530330017024']


In [35]:
######## Process data at Block Group Level
############

In [91]:
fname = 'bzone_carsvc_availability.csv'
# This file contains the information about level of car service availability and is used in the AssignCarSvcAvailability module.

#######################
# FIXME: update with values based on density or some other data
########################

# Extract something about average weight times or population from existing model. 
df = pd.read_csv(os.path.join(input_dir,fname))

_gdf_bg = gdf_bg[['geoid20']].copy()
_gdf_bg.rename(columns={'geoid20': 'Geo'}, inplace=True)
_gdf_bg_18 = _gdf_bg.copy()
_gdf_bg_18['Year'] = '2018'
_gdf_bg_18['CarSvcLevel'] = 'High'

_gdf_bg_50 = _gdf_bg_18.copy()
_gdf_bg_50['Year'] = '2050'

df = _gdf_bg_18.append(_gdf_bg_50)

# geo_df = pd.read_csv(os.path.join(output_dir,r'..\defs\geo.csv'))
# df = df[df['Geo'].isin(geo_df['Bzone'])]

# df = df[df['Geo'].astype('str').isin(geoid_list)]

df.to_csv(os.path.join(output_dir,fname), index=False)


In [93]:
# df[df['geoid20'] =]


# df

In [95]:

fname = 'bzone_dwelling_units.csv' 
# This file contains the number single-family, multi-family and group-quarter dwelling units and is used in the PredictHousing module.

# Extract from parcel file; assuming 0 group quarter units 
df = parcel_18.groupby('geoid20').sum()[['SFUNITS','MFUNITS']].reset_index()
df.rename(columns={'SFUNITS': 'SFDU', 'MFUNITS': 'MFDU', 'geoid20': 'Geo'}, inplace=True)
df['GQDU'] = 0
# df.iloc[0:6]['GQDU'] = 100     ################# FIXME: temp test

# Make sure we have values for all block groups
df['Year'] = '2018'

df_50 = parcel_18.groupby('geoid20').sum()[['SFUNITS','MFUNITS']].reset_index()
df_50.rename(columns={'SFUNITS': 'SFDU', 'MFUNITS': 'MFDU', 'geoid20': 'Geo'}, inplace=True)
df_50['GQDU'] = 0
# df.iloc[0:6]['GQDU'] = 100     ################# FIXME: temp test
df_50['Year'] = '2050'
# SFDU, fmdu, GQDU

df = df.append(df_50)

# df = df[df['Geo'].astype('str').isin(geoid_list)]

df.to_csv(os.path.join(output_dir,fname), index=False)

In [99]:
df[df[['SFDU','MFDU']].sum(axis=1) == 0]

,Geo,SFDU,MFDU,GQDU,Year
22,530330004043,0,0,0,2018
176,530330047022,0,0,0,2018
202,530330053031,0,0,0,2018
203,530330053032,0,0,0,2018
204,530330053041,0,0,0,2018
...,...,...,...,...,...
2203,530530729091,0,0,0,2050
2356,530610401004,0,0,0,2050
2369,530610407001,0,0,0,2050
2441,530610417043,0,0,0,2050


In [39]:

fname = 'bzone_employment.csv' 
# This file contains the total, retail and service employment by zone and is used in the LocateEmployment module.
df = pd.read_csv(os.path.join(input_dir,fname))

# 'Geo', 'Year', 'TotEmp', 'RetEmp', 'SvcEmp'
df = parcel_18.groupby('geoid20').sum()[['EMPRET_P','EMPSVC_P','EMPTOT_P']].reset_index()
df.rename(columns={'EMPRET_P': 'RetEmp', 'EMPSVC_P': 'SvcEmp', 'EMPTOT_P': 'TotEmp', 'geoid20': 'Geo'}, inplace=True)
df['Year'] = '2018'

df_50 = parcel_50.groupby('geoid20').sum()[['EMPRET_P','EMPSVC_P','EMPTOT_P']].reset_index()
df_50.rename(columns={'EMPRET_P': 'RetEmp', 'EMPSVC_P': 'SvcEmp', 'EMPTOT_P': 'TotEmp', 'geoid20': 'Geo'}, inplace=True)
df_50['Year'] = '2050'

df = df.append(df_50)

# df = df[df['Geo'].astype('str').isin(geoid_list)]

df.to_csv(os.path.join(output_dir,fname), index=False)


In [94]:
fname = 'bzone_hh_inc_qrtl_prop.csv'
#  This file contains the proportion of households in 1st, 2nd, 3rd, and 4th quartile of household income and is used in the PredictHousing module.
df = pd.read_csv(os.path.join(input_dir,fname))

def avg_income(year, run_dir):
    # Get average income from h5 files
    syn_h5 = h5py.File(os.path.join(run_dir, r'hh_and_persons.h5'), 'r')
    df_hh = pd.DataFrame()
    for col in syn_h5['Household'].keys():
        df_hh[col] = syn_h5['Household'][col][:]
    # df.loc[df['Year'] == '2018', ['HHIncomePC.2010','GQIncomePC.2010']] = df_hh['hhincome'].mean()


    # Join the block group to the household
    df_hh = df_hh.merge(parcel_18[['PARCELID','geoid20']], left_on='hhparcel', right_on='PARCELID')

    _df = pd.DataFrame(pd.qcut(df_hh['hhincome'], 4, labels=['1','2','3','4']))
    df_hh = df_hh.merge(_df, left_index=True, right_index=True)
    _df = df_hh.pivot_table(index='geoid20', columns='hhincome_y', values='hhexpfac', aggfunc='sum')

    df_sum = pd.DataFrame(_df[['1','2','3','4']].sum(axis=1)).reset_index()
    df_sum.rename(columns={0: 'total_hh', 'geoid20': 'Geo'}, inplace=True)
    _df = df_sum.merge(_df, left_on='Geo', right_index=True)
    for i in [1,2,3,4]:
        _df['HhPropIncQ'+str(i)] = _df[str(i)]/_df['total_hh']

    # Make sure all block groups are available
    full_gdf_bg = read_from_sde(connection_string, 'blockgrp2020', version, crs=crs, is_table=False)
    missing_bg = full_gdf_bg[~full_gdf_bg['geoid20'].isin(_df['Geo'])][['geoid20']]
    # print(missing_bg)
    temp_df = _df.iloc[:len(missing_bg)].copy()
    # Set income distributions for empty block groups to uniform distribution
    temp_df[['HhPropIncQ1','HhPropIncQ2','HhPropIncQ3', 'HhPropIncQ4']] = 0.25
    temp_df['Geo'] = missing_bg['geoid20'].values
    _df = _df.append(temp_df)

    # Filter for rows in the geo file
    #### FIXME: This should depend on the final list of zones to be included
    geo_df = pd.read_csv(os.path.join(output_dir,r'..\defs\geo.csv'))
    _df = _df[_df['Geo'].isin(geo_df['Bzone'].astype('str'))]
    
    # _df = _df[_df['Geo'].astype('str').isin(geoid_list)]

    _df['Year'] = year

    return _df[df.columns]

df = avg_income('2018', r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2018\new_emp')
df_50 = avg_income('2050', r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2050\lodes')
df = df.append(df_50)
df.to_csv(os.path.join(output_dir,fname), index=False)

C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [78]:
df[df['Geo'] == '530330053041']
# df

,Geo,Year,HhPropIncQ1,HhPropIncQ2,HhPropIncQ3,HhPropIncQ4
0,530330053041,2018,0.25,0.25,0.25,0.25
0,530330053041,2050,0.25,0.25,0.25,0.25


In [42]:
# run_dir = r'R:\e2projects_two\SoundCast\Inputs\dev\landuse\2018\new_emp'
# syn_h5 = h5py.File(os.path.join(run_dir, r'hh_and_persons.h5'), 'r')
# df_hh = pd.DataFrame()
# for col in syn_h5['Household'].keys():
#     df_hh[col] = syn_h5['Household'][col][:]
# # df.loc[df['Year'] == '2018', ['HHIncomePC.2010','GQIncomePC.2010']] = df_hh['hhincome'].mean()


# # Join the block group to the household
# df_hh = df_hh.merge(parcel_18[['PARCELID','geoid20']], left_on='hhparcel', right_on='PARCELID')

# _df = pd.DataFrame(pd.qcut(df_hh['hhincome'], 4, labels=['1','2','3','4']))
# df_hh = df_hh.merge(_df, left_index=True, right_index=True)
# _df = df_hh.pivot_table(index='geoid20', columns='hhincome_y', values='hhexpfac', aggfunc='sum')

# df_sum = pd.DataFrame(_df[['1','2','3','4']].sum(axis=1)).reset_index()
# df_sum.rename(columns={0: 'total_hh', 'geoid20': 'Geo'}, inplace=True)
# _df = df_sum.merge(_df, left_on='Geo', right_index=True)
# for i in [1,2,3,4]:
#     _df['HhPropIncQ'+str(i)] = _df[str(i)]/_df['total_hh']

# # Make sure all block groups are available
# # full_gdf_bg = read_from_sde(connection_string, 'blockgrp2020', version, crs=crs, is_table=False)
# # missing_bg = full_gdf_bg[~full_gdf_bg['geoid20'].isin(_df['Geo'])]['geoid20']
# # print(missing_bg)
# # temp_df = _df.iloc[:len(missing_bg)].copy()
# # # Set income distributions for empty block groups to uniform distribution
# # temp_df[['HhPropIncQ1','HhPropIncQ2','HhPropIncQ3', 'HhPropIncQ4']] = 0.25
# # temp_df['Geo'] = missing_bg['geoid20'].values
# # _df = _df.append(temp_df)

# syn_h5.close()
# 


In [43]:
# Filter for rows in the geo file
#### FIXME: This should depend on the final list of zones to be included
# geo_df = pd.read_csv(os.path.join(output_dir,r'..\defs\geo.csv'))
# _df = _df[_df['Geo'].isin(geo_df['Bzone']geo_df['Bzone'].astype('str'))]

In [44]:
# _df[_df['Geo'].isin(geo_df['Bzone'].astype('str'))]

In [45]:
fname = 'bzone_lat_lon.csv' 
# This file contains the latitude and longitude of the centroid of the zone and is used in the LocateEmployment module.
df = pd.read_csv(os.path.join(input_dir,fname))

# Get from intptlat and intpltlon
###########################
#### FIXME: ensure these are the correct values, not sure how they are calculated or their source
###########################
gdf_bg_18 = gdf_bg.copy()
gdf_bg_18.rename(columns={'intptlat': 'Latitude', 'intptlon': 'Longitude', 'geoid20': 'Geo'}, inplace=True)
gdf_bg_18['Year'] = '2018'

gdf_bg_50 = gdf_bg_18.copy()
gdf_bg_50['Year'] = '2050'

df = gdf_bg_18.append(gdf_bg_50)[['Geo', 'Year', 'Latitude', 'Longitude']]

# df = df[df['Geo'].astype('str').isin(geoid_list)]

df.to_csv(os.path.join(output_dir,fname), index=False)

In [46]:
# df

In [100]:
fname = 'bzone_network_design.csv'
df = pd.read_csv(os.path.join(input_dir,fname))
#  This file contains the intersection density in terms of pedestrian-oriented intersections having four or more legs per square mile and is used in the Calculate4DMeasures module.

# d3bpo4 https://www.epa.gov/sites/default/files/2021-06/documents/epa_sld_3.0_technicaldocumentationuserguide_may2021.pdf
# Intersection density in terms of pedestrian-oriented
# intersections having four or more legs per square mile 


#######
## Let's just use pandana lib for this
def calculate_intersections(run_dir, year):
    nodes = pd.read_csv(os.path.join(run_dir, r'inputs\base_year\all_streets_nodes.csv'), 
                    index_col='node_id')
    links = pd.read_csv(os.path.join(run_dir, r'inputs\base_year\all_streets_links.csv'), 
                    index_col=None)

    # get rid of circular links
    links = links.loc[(links.from_node_id != links.to_node_id)]

    # assign impedance
    imp = pd.DataFrame(links.Shape_Length)
    imp = imp.rename(columns = {'Shape_Length':'distance'})
    links[['from_node_id','to_node_id']] = links[['from_node_id','to_node_id']].astype('int') 

    net = pdna.network.Network(nodes.x, nodes.y, links.from_node_id, links.to_node_id, imp)
    all_nodes = pd.DataFrame(net.edges_df['from'].append(net.edges_df.to), columns = ['node_id'])

    # get the frequency of each node, which is the number of intersecting ways
    intersections_df = pd.DataFrame(all_nodes.node_id.value_counts())
    intersections_df = intersections_df.rename(columns = {'node_id' : 'edge_count'})
    intersections_df.reset_index(0, inplace = True)
    intersections_df = intersections_df.rename(columns = {'index' : 'node_ids'})

    # add a column for each way count
    intersections_df['nodes4'] = np.where(intersections_df['edge_count']>3, 1, 0)

    df = nodes.merge(intersections_df, left_index=True, right_on='node_ids')
    # filter for all 4-way intersections
    df = df[df['nodes4'] > 0]

    # Convert to geopandas dataframe
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.x, df.y))
    crs = {'init' : 'EPSG:2285'}
    gdf.crs = crs

    gdf = gpd.sjoin(gdf, gdf_bg)

    df = gdf.groupby('geoid20').sum()[['nodes4']]

    # Join with other block groups not including any 4-way intersections
    df = gdf_bg.merge(df, on='geoid20', how='left')[['geoid20', 'nodes4']]
    df['nodes4'].fillna(0, inplace=True)
    df.rename(columns={'nodes4': 'D3bpo4', 'geoid20': 'Geo'}, inplace=True)
    df['Year'] = year

    # #### FIXME: This should depend on the final list of zones to be included
    # geo_df = pd.read_csv(os.path.join(output_dir,r'..\defs\geo.csv'))
    # df = df[df['Geo'].isin(geo_df['Bzone'])]

    # df = df[df['Geo'].astype('str').isin(geoid_list)]

    return df

In [101]:
df_18 = calculate_intersections(run_dir_18, '2018')
df_50 = calculate_intersections(run_dir_50, '2050')
df = df_18.append(df_50)
df.to_csv(os.path.join(output_dir,fname), index=False)

C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [109]:
# df[df['Geo'] == '530330012022']
# # gdf_bg[gdf_bg['geoid20'] == '530330012022']
# [print(i) for i in df['Geo'].unique()]
# df[df['Geo'] == 'NA']

In [50]:
fname = 'bzone_parking.csv' 
df = pd.read_csv(os.path.join(input_dir,fname))
df
# This file contains the parking information and is used in the AssignParkingRestrictions module.

# PkgSpacesPerSFDU: Average number of free parking spaces available to residents of single-family dwelling units
# PkgSpacesPerMFDU: Average number of free parking spaces available to residents of multifamily dwelling units
# PkgSpacesPerGQ: Average number of free parking spaces available to group quarters residents
# PropWkrPay: Proportion of workers who pay for parking
# PropCashOut: Proportions of workers paying for parking in a cash-out-buy-back program
# PkgCost: Average daily cost for long-term parking (e.g. paid on monthly basis)

### FIXME: see if we can get this info; shouldn't come from parcel data becasue this is limited to on-street parking

# df = pd.read_csv(os.path.join(input_dir,fname))
# df
# should mostly be available from parcels file

# _df['avg_dy_price'] = _df['PARKDY_P']/_df['PPRICDYP']
# _df['avg_hr_price'] = _df['PARKHR_P']/_df['PPRICHRP']

def get_parking_restrictions(year):
##### FIXME
#############
    _df = parcel_18.groupby('geoid20').sum()[['PARKDY_P','PARKHR_P','SFUNITS','MFUNITS']]
    _df = _df.reset_index()
    _df['PkgSpacesPerSFDU'] = 2    # Set equal to 2 for now
    _df['PkgSpacesPerMFDU'] = 0.5    # Set equal to 0.5 for now
    _df['PkgSpacesPerGQ'] = 0.5     # set to same as MF ?

    df.columns
    # 
    # PropWrkPay
    # Is a variable in daysim but we do not populate it
    _df_mean = parcel_18.groupby('geoid20').mean()[['PPRICDYP','PPRICHRP']]
    _df_mean = _df_mean.reset_index()
    _df = _df.merge(_df_mean, on='geoid20', how='left')
    _df['paid'] = 0
    _df.loc[_df['PPRICHRP'] > 0, 'paid'] = 1
    _df['PropNonWrkTripPay'] = _df['paid']
    _df['PropWkrPay'] = _df['paid']
    _df['PropCashOut'] = 0
    _df['PkgCost.2010'] = _df['PPRICDYP']/100

    _df.rename(columns={'geoid20': 'Geo', }, inplace=True)
    _df['Year'] = year

    return _df

df18 = get_parking_restrictions('2018')[df.columns]
df50 = get_parking_restrictions('2050')[df.columns]
df = df18.append(df50)

# dfparcel = df[df['Geo'].astype('str').isin(geoid_list)]
# df = df[df['Geo'].astype('str').isin(geoid_list)]

df.to_csv(os.path.join(output_dir,fname), index=False)


In [54]:
# df[df['PropNonWrkTripPay']>0]

In [55]:
# _df[_df['PARKHR_P'] >0]

In [52]:

fname = 'bzone_travel_demand_mgt.csv'
_df = pd.read_csv(os.path.join(input_dir,fname))
# This file contains the information about workers and households participating in demand management programs and is used in the AssignDemandManagement module.

def get_tdm(year):
    _df = parcel_18.groupby('geoid20').count()[['PARCELID']].reset_index()
    _df = _df.reset_index()
    # could use output of transit pass ownership model or apply some off model thing based on survey data
    # FIXME: update this from some other attribute

    # EcoProp: Proportion of workers working in Bzone who participate in strong employee commute options program
    # ImpProp: Proportion of households residing in Bzone who participate in strong individualized marketing program
    _df['EcoProp'] = 0
    _df['ImpProp'] = 0
    _df.rename(columns={'geoid20': 'Geo'}, inplace=True)
    _df['Year'] = year

    return _df

df18 = get_tdm('2018')[_df.columns]
df50 = get_tdm('2050')[_df.columns]
df = df18.append(df50)

# df = df[df['Geo'].astype('str').isin(geoid_list)]

df.to_csv(os.path.join(output_dir,fname), index=False)

In [53]:
df

,Geo,Year,EcoProp,ImpProp
0,530330001011,2018,0,0
1,530330001012,2018,0,0
2,530330001013,2018,0,0
3,530330001021,2018,0,0
4,530330001022,2018,0,0
...,...,...,...,...
2923,530619400021,2050,0,0
2924,530619400022,2050,0,0
2925,530619400023,2050,0,0
2926,530619900020,2050,0,0


In [140]:

fname = 'bzone_unprotected_area.csv'
# This file contains the information about unprotected (i.e., developable) area within the zone and is used in the Calculate4DMeasures module.
_df = pd.read_csv(os.path.join(input_dir,fname))

# get developable area from UGA boundaries
# ElmerGeo.DBO.tod_prcl_uga
# gdf_shp = read_from_sde(connection_string, 'tod_prcl_uga', version, crs=crs, is_table=False)
# gdf_shp = read_from_sde(connection_string, 'urban_growth_area', version, crs=crs, is_table=False)
# gdf_shp = gdf_shp.dissolve()
# gdf_shp['in_uga'] = 1

def get_unprotected_area(year, _df):
    # Use the block group shapefile with no water because we are calculating buildable area
    gdf_bg = read_from_sde(connection_string, 'blockgrp2020_nowater', version, crs=crs, is_table=False)
    gdf_bg['total_area'] = gdf_bg.area

    # Add rural/town/urban definitions using regional geographies
    rg_shp = read_from_sde(connection_string, 'regional_geographies_preferred_alternative', version, crs=crs, is_table=False)

    # Load the UGA bounds
    # Create these defintions based on regional geographies and UGA?
    # UrbanArea: Area that is Urban and unprotected (i.e. developable) within the zone (Acres)
    # TownArea: Area that is Town and unprotected within the zone (Acres)
    # RuralArea: Area that is Rural and unprotected within the zone (Acres)
    rg_shp['rg_propose_pa'].unique()
    # rg_shp.columns
    rg_dict = {'CitiesTowns': 'TownArea',
                'Core': 'UrbanArea',
                'UU': 'UrbanArea',
                'Metro': 'UrbanArea',
                'HCT': 'UrbanArea',
                'Rural': 'RuralArea'}
    rg_shp['urban_rural'] = rg_shp['rg_propose_pa'].map(rg_dict)

    join_rg_gdf = gpd.overlay(gdf_bg, rg_shp, how='intersection')

    join_rg_gdf['rg_area'] = join_rg_gdf.area

    df = join_rg_gdf.pivot_table(index='geoid20', columns='urban_rural', aggfunc='sum', values='rg_area').fillna(0)
    df = df.reset_index()
    # df.drop('urban_rural', axis=1, inplace=True)

    # Make sure all block groups are available
    # Since we used the no_water version, there are some locations that were excluded
    full_gdf_bg = read_from_sde(connection_string, 'blockgrp2020', version, crs=crs, is_table=False)
    missing_bg = full_gdf_bg[~full_gdf_bg['geoid20'].isin(df['geoid20'])]
    # print(missing_bg['geoid20'])
    temp_df = df.iloc[:len(missing_bg)].copy()

    # assign average acreage for all missing block groups (distributing equally across area types)
    temp_df[['UrbanArea','TownArea','RuralArea']] = full_gdf_bg['acres'].mean()/3
    temp_df['geoid20'] = missing_bg['geoid20'].values
    df = df.append(temp_df)

    # Convert to acres
    df[['RuralArea','TownArea','UrbanArea']] = df[['RuralArea','TownArea','UrbanArea']]/43560.0

    df['Year'] = year
    df.rename(columns={'geoid20': 'Geo'}, inplace=True)

    return df[_df.columns]

df_18 = get_unprotected_area('2018', _df)


C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [145]:
df_50 = get_unprotected_area('2050', _df)
df = df_18.append(df_50)
df.to_csv(os.path.join(output_dir,fname), index=False)

C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [146]:
df[df[['UrbanArea','TownArea','RuralArea']].sum(axis=1) == 0]

urban_rural,Geo,Year,UrbanArea,TownArea,RuralArea


In [130]:
full_gdf_bg = read_from_sde(connection_string, 'blockgrp2020', version, crs=crs, is_table=False)


C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


1502.6688120293613

In [56]:
fname = 'bzone_urban-mixed-use_prop.csv'
# This file contains the target proportion of households located in mixed-used neighborhoods in zone and is used in the CalculateUrbanMixMeasure module.

_df = pd.read_csv(os.path.join(input_dir,fname))

# Not sure how to define this; from parcels aggregate to BG and consider any zone with employment and households as mixed use?


def get_mixed_use(parcel_df, year, _df):
    
    # parcel_df = parcel_18.copy()
    parcel_df['total_units'] = parcel_df['SFUNITS']+parcel_df['MFUNITS']
    parcel_df['mixed_use'] = 0
    parcel_df.loc[parcel_df['SFUNITS'] < parcel_df['total_units'], 'mixed_use'] = 1

    # Calculate
    parcel_df['wt_mixed_use'] = parcel_df['mixed_use']*parcel_df['total_units']

    df = parcel_df.groupby('geoid20').sum()[['total_units','wt_mixed_use']].reset_index()
    df['MixUseProp'] = df['wt_mixed_use']/df['total_units']
    df['MixUseProp'].fillna(0, inplace=True)

    df['Year'] = year
    df.rename(columns={'geoid20': 'Geo'}, inplace=True)

    return df[_df.columns]

df_18 = get_mixed_use(parcel_18.copy(), '2018', _df)
df_50 = get_mixed_use(parcel_50.copy(), '2050', _df)
df = df_18.append(df_50)

# df = df[df['Geo'].astype('str').isin(geoid_list)]

df.to_csv(os.path.join(output_dir,fname), index=False)

In [57]:
# df

In [180]:
fname = 'bzone_urban-town_du_proportions.csv'
# This file contains proportion of Single-Family, Multi-Family and Group Quarter dwelling units within the urban portion of the zone and is used in the AssignLocTypes module.

# from parcels file with land use types associated


_df = pd.read_csv(os.path.join(input_dir,fname))
_df

rg_shp = read_from_sde(connection_string, 'regional_geographies_preferred_alternative', version, crs=crs, is_table=False)


def get_du_proportions(rg_shp, _parcel_df, year, _df):
    # Note that we convert results to INT so there will never be a partial result
    # Either an area is majority urban or rural; floats were causing issues when running scenarios
    gdf = gpd.sjoin(_parcel_df, rg_shp, how='left')

    rg_dict = {'CitiesTowns': 'TownArea',
                'Core': 'UrbanArea',
                'UU': 'UrbanArea',
                'Metro': 'UrbanArea',
                'HCT': 'UrbanArea',
                'Rural': 'RuralArea'}
    gdf['urban_rural'] = gdf['rg_propose_pa'].map(rg_dict)

    df = gdf.pivot_table(index='geoid20', columns='urban_rural', values=['MFUNITS','SFUNITS'], aggfunc='sum').fillna(0).reset_index()
    
    for unit_type in ['SFUNITS','MFUNITS']:
        df['PropTown'+unit_type[0:2]+'DU'] = df[unit_type]['TownArea']/df[unit_type].sum(axis=1).fillna(0).astype('int')
        df['PropUrban'+unit_type[0:2]+'DU'] = df[unit_type]['UrbanArea']/df[unit_type].sum(axis=1).fillna(0).astype('int')

    ######################
    # FIXME: setting this match format from RVMPO, but not sure why it doesn't work as it
    ######################
    # df['PropUrbanSFDU'] = (df['SFUNITS']['UrbanArea']/df['SFUNITS'].sum(axis=1)).fillna(0)
    # df['PropUrbanSFDU'] = 1
    # df['PropUrbanMFDU'] = 1
    # df['PropTownSFDU'] = 0
    # df['PropTownMFDU'] = 0
    df['PropTownGQDU'] = 0
    df['PropUrbanGQDU'] = 0
    df['Year'] = year
    df.rename(columns={'geoid20': 'Geo'}, inplace=True)
    df.fillna(0, inplace=True)
    df.columns = df.columns.droplevel(1)


    return df[_df.columns]

df_18 = get_du_proportions(rg_shp, parcel_18_gdf, '2018', _df)
df_50 = get_du_proportions(rg_shp, parcel_50_gdf, '2050', _df)
df = df_18.append(df_50)

# df = df[df['Geo'].astype('str').isin(geoid_list)]

df.to_csv(os.path.join(output_dir,fname), index=False)


ProgrammingError: (pyodbc.ProgrammingError) ('42000', '[42000] [Microsoft][ODBC SQL Server Driver][SQL Server]Failed to initialize the Common Language Runtime (CLR) v4.0.30319 with HRESULT 0x80004001. You may fix the problem and try again later. (6511) (SQLExecDirectW)')
[SQL: select *, Shape.STAsText() as geometry from regional_geographies_preferred_alternative]
(Background on this error at: http://sqlalche.me/e/14/f405)

In [179]:
# df = pd.read_csv(r'C:\Workspace\VisionEval\models\VERSPM\inputs\bzone_urban-town_du_proportions.csv')
# col_list = ['PropUrbanSFDU','PropUrbanMFDU','PropUrbanGQDU','PropTownSFDU','PropTownMFDU','PropTownGQDU']
# df[col_list] = df[col_list].astype('int')
# df.to_csv(r'C:\Workspace\VisionEval\models\VERSPM\inputs\bzone_urban-town_du_proportions.csv', index=False)

In [ ]:
df

,Geo,Year,PropUrbanSFDU,PropUrbanMFDU,PropUrbanGQDU,PropTownSFDU,PropTownMFDU,PropTownGQDU
0,530330001011,2018,1,1,0,0,0,0
1,530330001012,2018,1,1,0,0,0,0
2,530330001013,2018,1,1,0,0,0,0
3,530330001021,2018,1,1,0,0,0,0
4,530330001022,2018,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...
2919,530619400021,2050,1,1,0,0,0,0
2920,530619400022,2050,1,1,0,0,0,0
2921,530619400023,2050,1,1,0,0,0,0
2922,530619900020,2050,1,1,0,0,0,0


In [ ]:
# rg_shp = read_from_sde(connection_string, 'regional_geographies_preferred_alternative', version, crs=crs, is_table=False)
# _parcel_df = parcel_18_gdf.copy()
# gdf = gpd.sjoin(_parcel_df, rg_shp, how='left')

# rg_dict = {'CitiesTowns': 'TownArea',
#             'Core': 'UrbanArea',
#             'UU': 'UrbanArea',
#             'Metro': 'UrbanArea',
#             'HCT': 'UrbanArea',
#             'Rural': 'RuralArea'}
# gdf['urban_rural'] = gdf['rg_propose_pa'].map(rg_dict)

In [ ]:
# df = gdf.pivot_table(index='geoid20', columns='urban_rural', values=['MFUNITS','SFUNITS'], aggfunc='sum').fillna(0).reset_index()

In [ ]:
# df

In [ ]:

# gdf = gpd.sjoin(_parcel_df, rg_shp, how='left')
# _parcel_df = parcel_18_gdf.copy()
# _parcel_df = _parcel_df[_parcel_df['PARCELID'] != 0] 
# gdf = gpd.sjoin(_parcel_df, rg_shp, how='left')
# rg_dict = {'CitiesTowns': 'TownArea',
#         'Core': 'UrbanArea',
#         'UU': 'UrbanArea',
#         'Metro': 'UrbanArea',
#         'HCT': 'UrbanArea',
#         'Rural': 'RuralArea'}
# gdf['urban_rural'] = gdf['rg_propose_pa'].map(rg_dict)

# df = gdf.pivot_table(index='geoid20', columns='urban_rural', values=['MFUNITS','SFUNITS'], aggfunc='sum').fillna(0).reset_index()

# for lu_type in ['Rural','Town','Urban']:
#     df['tot_'+lu_type] = df['MFUNITS'][lu_type+'Area']+df['SFUNITS'][lu_type+'Area']
#     df['Prop'+lu_type+'SFDU'] = df['SFUNITS'][lu_type+'Area']/df['tot_'+lu_type]
#     df['Prop'+lu_type+'MFDU'] = df['MFUNITS'][lu_type+'Area']/df['tot_'+lu_type]

In [ ]:
# rewrite geo.csv to only include block groups with some dwelling units
# missing_bg
# missing_bg = [530330053041,530330228033,530619900020,530339901000,
#             530350903001,530359901000,530530726035,530530729031,
#             530530729072,530530729082,530610401004,530619900020,530619901000]
# fname = 'geo.csv'
# df = pd.read_csv(os.path.join(output_dir,'..\defs',fname))
# df = df[~df['Bzone'].isin(missing_bg)]
# df.to_csv(os.path.join(output_dir,fname), index=False)

# M Area
Regional totals


In [ ]:
########### Optional files, not sure if they need to be included with zeros or not

######### FIXME: 
######### create empty files with na values based on existing
#########
######### do something like load the existing file, fill with our region name and model years, and add empty null values

'marea_base_year_dvmt.csv'
# This file is used to specify to adjust the dvmt growth factors and is optional (only needed if user wants to modify the values). The file is used in the Initialize (VETravelPerformance), CalculateBaseRoadDvmt and CalculateFutureRoadDvmt modules.

'marea_congestion_charges.csv' 
# This file is used to specify the charges of vehicle travel for different congestion levels and is optional. The file is used in the Initialize (VETravelPerformance) and CalculateRoadPerformance modules.

'marea_dvmt_split_by_road_class.csv' 
# This file is used to specify the dvmt split for different road classes and is optional. The file is used in the Initialize (VETravelPerformance) and CalculateBaseRoadDvmt modules.

'marea_operations_deployment.csv' 
# This file is used to specify the proportion of dvmt affected by operations for different road classes and is optional. The file is used in the Initialize (VETravelPerformance) and CalculateRoadPerformance modules.

'marea_transit_ave_fuel_carbon_intensity.csv' 
# This file is used to specify the average carbon intensity of fuel used by transit and is optional. The file is used in the Initialize (VETravelPerformance) module.

'marea_transit_biofuel_mix.csv' 
#This file is used to specify the biofuel used by transit and is optional. The file is used in the Initialize (VETravelPerformance) and CalculateCarbonIntensity modules.

'marea_transit_fuel.csv' 
# This file is used to specify the transit fuel proportions and is optional. The file is used in the Initialize (VETravelPerformance) and CalculateCarbonIntensity modules.

'marea_transit_powertrain_prop.csv' 
# This file is used to specify the mixes of transit vehicle powertrains and is optional. The file is used in the Initialize (VETravelPerformance) and CalculatePtranEnergyAndEmissions modules.

for fname in ['marea_transit_powertrain_prop.csv',
            'marea_base_year_dvmt.csv',
            'marea_congestion_charges.csv',
            'marea_dvmt_split_by_road_class.csv',
            'marea_operations_deployment.csv',
            'marea_transit_ave_fuel_carbon_intensity.csv',
            'marea_transit_biofuel_mix.csv',
            'marea_transit_fuel.csv',
            'marea_transit_powertrain_prop.csv']:
    df = pd.read_csv(os.path.join(input_dir,fname))
    # print(fname)
    df['Geo'] = regional_geo
    if 'Year' in df.columns:
        df['Year'] = df['Year'].replace({2010: 2018, 2038: 2050})
    df.to_csv(os.path.join(output_dir,fname), index=False)

In [ ]:
fname = 'marea_lane_miles.csv' 
# This file contains inputs on the numbers of freeway lane-miles and arterial lane-miles and is used in the AssignRoadMiles module.

# FwyLaneMi: Lane-miles of roadways functionally classified as freeways or expressways in the urbanized portion of the metropolitan area
# ArtLaneMi: Lane-miles of roadways functionally classified as arterials (but not freeways or expressways) in the urbanized portion of the metropolitan area

def load_network_summary(filepath):
    """Load network-level results using a standard procedure. """
    df = pd.read_csv(filepath)

    # Congested network components by time of day
    df.columns

    # Get freeflow from 20to5 period

    # Exclude trips taken on non-designated facilities (facility_type == 0)
    # These are artificial (weave lanes to connect HOV) or for non-auto uses 
    df = df[df['data3'] != 0]    # data3 represents facility_type

    # calculate total link VMT and VHT
    df['VMT'] = df['@tveh']*df['length']
    df['VHT'] = df['@tveh']*df['auto_time']/60

    # Define facility type
    df.loc[df['data3'].isin([1,2]), 'facility_type'] = 'highway'
    df.loc[df['data3'].isin([3,4,6]), 'facility_type'] = 'arterial'
    df.loc[df['data3'].isin([5]), 'facility_type'] = 'connector'

    # Calculate delay
    # Select links from overnight time of day
    delay_df = df.loc[df['tod'] == '20to5'][['ij','auto_time']]
    delay_df.rename(columns={'auto_time':'freeflow_time'}, inplace=True)

    # Merge delay field back onto network link df
    df = pd.merge(df, delay_df, on='ij', how='left')

    # Calcualte hourly delay
    df['total_delay'] = ((df['auto_time']-df['freeflow_time'])*df['@tveh'])/60    # sum of (volume)*(travtime diff from freeflow)

    df['county'] =df['@countyid'].map({33: 'King',
                                      35: 'Kitsap',
                                      53: 'Pierce',
                                      61: 'Snohomish'})
    
    return df

def get_lane_miles(year, run_dir):
    """Note: only want lane miles iwthin URBANZINED PORTION"""
    df_network = load_network_summary(os.path.join(run_dir,r'outputs\network','network_results.csv'))
    # Select mid-day network
    gdf = df_network[df_network['tod'] == '10to14']
    gdf['Lane Miles'] = gdf['length']*gdf['num_lanes']

    ul3_dict = {
        0: 'Rail/Walk/Ferry',
        1: 'Freeway',
        2: 'Expressway',
        3: 'Urban Arterial',
        4: 'One-way Arterial',
        5: 'Centroid Connector',
        6: 'Rural Arterial'
    }

    gdf['Facility Group'] = gdf['data3'].map(ul3_dict)
    df = gdf.groupby(['Facility Group','data3']).sum()[['Lane Miles']].sort_values('data3').reset_index()

    _df = pd.DataFrame([df[df['Facility Group'].isin(['Freeway','Expressway'])]['Lane Miles'].sum(),
                    df[df['Facility Group'].isin(['Urban Arterial','Rural Arterial', 'One-way arterial'])]['Lane Miles'].sum()]).T
    _df.columns = ['FwyLaneMi', 'ArtLaneMi']
    _df['Year'] = year
    
    _df['Geo'] = regional_geo
    
    return _df

regional_geo = 'PSRC'
df18 = get_lane_miles('2018', r'L:\RTP_2022\final_runs\sc_2018_rtp_final\soundcast')
df50 = get_lane_miles('2050', r'L:\RTP_2022\final_runs\sc_rtp_2050_constrained_final\soundcast')

df = df18.append(df50)
df.to_csv(os.path.join(output_dir,fname), index=False)

C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [ ]:
fname = 'marea_speed_smooth_ecodrive.csv'
# This input file supplies information of deployment of speed smoothing and ecodriving by road class and vehicle type and is used in the CalculateMpgMpkwhAdjustments module.

# Set to zero for now
# FwySmooth:Fractional deployment of speed smoothing traffic management on freeways, where 0 is no deployment and 1 is the full potential fuel savings
# ArtSmooth: Fractional deployment of speed smoothing traffic management on arterials, where 0 is no deployment and 1 is the full potential fuel savings
# LdvEcoDrive: Eco-driving penetration for light-duty vehicles; the fraction of vehicles from 0 to 1
# HvyTrkEcoDrive: Eco-driving penetration for heavy-duty vehicles; the fraction of vehicles from 0 to 1

df = pd.DataFrame([0,0,0,0]).T
df.columns = ['FwySmooth','ArtSmooth','LdvEcoDrive','HvyTrkEcoDrive']
df['Year'] = '2018'
df['Geo'] = regional_geo


df50 = pd.DataFrame([0,0,0,0]).T
df50.columns = ['FwySmooth','ArtSmooth','LdvEcoDrive','HvyTrkEcoDrive']
df50['Year'] = '2050'
df50['Geo'] = regional_geo

df = df.append(df50)
df.to_csv(os.path.join(output_dir,fname), index=False)

In [121]:
# df = pd.read_csv(os.path.join(run_dir,r'inputs\scenario\networks\shapefiles\AM\AM_transit_segments.csv'))
# df[df['LineID'] == 118002]

,Unnamed: 0,route_id,INode,JNode,order,stop_number,is_stop,i,j,length,ttf,stop_to_stop_distance,loop_index,seg_id,ij,transit_mode,LineID
0,0,118002,112850,112846,1,1,1,112850,112846,0.075916,11,0.321659,1,118002-112850-112846,112850-112846,b,118002
1,1,118002,112846,112765,2,1,0,112846,112765,0.010751,11,0.321659,1,118002-112846-112765,112846-112765,b,118002
2,2,118002,112765,112651,3,1,0,112765,112651,0.019321,11,0.321659,1,118002-112765-112651,112765-112651,b,118002
3,3,118002,112651,111598,4,1,0,112651,111598,0.151089,11,0.321659,1,118002-112651-111598,112651-111598,b,118002
4,4,118002,111598,111608,5,1,0,111598,111608,0.064581,11,0.321659,1,118002-111598-111608,111598-111608,b,118002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,134,118002,65169,65177,135,55,1,65169,65177,0.063525,11,0.246699,1,118002-65169-65177,65169-65177,b,118002
135,135,118002,65177,65188,136,55,0,65177,65188,0.059300,11,0.246699,1,118002-65177-65188,65177-65188,b,118002
136,136,118002,65188,65222,137,55,0,65188,65222,0.123874,11,0.246699,1,118002-65188-65222,65188-65222,b,118002
137,137,118002,65222,65372,138,56,1,65222,65372,0.128811,11,0.166294,1,118002-65222-65372,65222-65372,b,118002


In [98]:

fname = 'marea_transit_service.csv' 
# This file contains annual revenue-miles for different transit modes for metropolitan area and is used in the AssignTransitService module.

# Get from Stefan's transit tool
# import transit_service_analystk as tsa

# path = r'R:/e2projects_two/Angela/transit_routes_2018/latest_combined'
# transit_analyst = tsa.load_gtfs(path, '20180417', 0, 1680)

# geo_df = transit_analyst.get_lines_gdf()

# DRRevMi: Annual revenue-miles of demand-responsive public transit service
# VPRevMi: Annual revenue-miles of van-pool and similar public transit service
# MBRevMi: Annual revenue-miles of standard bus public transit service
# RBRevMi: Annual revenue-miles of rapid-bus and commuter bus public transit service
# MGRevMi: Annual revenue-miles of monorail and automated guideway public transit service
# SRRevMi: Annual revenue-miles of streetcar and trolleybus public transit service
# HRRevMi: Annual revenue-miles of light rail and heavy rail public transit service
# CRRevMi: Annual revenue-miles of commuter rail, hybrid rail, cable car, and aerial tramway public transit service

# GTFS Routes Types:
# 0 - Tram, Streetcar, Light rail. Any light rail or street level system within a metropolitan area.
# 1 - Subway, Metro. Any underground rail system within a metropolitan area.
# 2 - Rail. Used for intercity or long-distance travel.
# 3 - Bus. Used for short- and long-distance bus routes.
# 4 - Ferry. Used for short- and long-distance boat service.
# 5 - Cable tram. Used for street-level rail cars where the cable runs beneath the vehicle, e.g., cable car in San Francisco.
# 6 - Aerial lift, suspended cable car (e.g., gondola lift, aerial tramway). Cable transport where cabins, cars, gondolas or open chairs are suspended by means of one or more cables.
# 7 - Funicular. Any rail system designed for steep inclines.
# 11 - Trolleybus. Electric buses that draw power from overhead wires using poles.
# 12 - Monorail. Railway in which the track consists of a single rail or a beam.

#########################

# FIXME! need to be scaled up to get revenue vehicle miles; on the order of 60+M for Kc Metro
# https://www.transit.dot.gov/sites/fta.dot.gov/files/transit_agency_profile_doc/2018/00010-00001.pdf

# FIXME! how to code ferry? coding as catch-all commuter/hybrid rail plus cable car and aerial tramway

# FIXME! get a GTFS for future year or pull form network?

#########################

# route_type_map = {
#     0: 'HRRevMi',   # streetcar/light rail -> light/heavy rail
#     1: 'HRRevMi',   # subway/metro -> light/heavy rail
#     2: 'CRRevMi',   # commuter rail plus cable car, aerial tram
#     3: 'MBRevMi',   # bus -> standard bus
#     4: 'CRRevMi',  # ferry -> commuter rail plus cable car, aerial tram
#     5: 'CRRevMi',   # cable tram ->  ' '
#     6: 'CRRevMi',   # aerial lift -> ' '
#     7: 'CRRevMi',   # funicular -> ' '
#     11: 'SRRevMi',  # trolleybus -> streetcar and trolleybus service
#     12: 'MGRevMi',  # monorail
# }

# annualize weekday service?
annual_factor = 300

# # Get revenue miles by multiplying daily vehicles times miles traveled
# run_dir = r'C:\Workspace\sc_2018_rtp_final\soundcast'
# df_transit = gpd.read_file(os.path.join(run_dir,r'inputs\scenario\networks\shapefiles\AM\AM_edges.shp'))

# df = pd.read_csv(os.path.join(run_dir,r'inputs\scenario\networks\shapefiles\AM\AM_transit_segments.csv'))
# df_headways = pd.read_csv(os.path.join(run_dir,r'inputs\scenario\networks\headways.csv'))
# hdwy_cols = df_headways.columns.drop(['Unnamed: 0','LineID','id'])
# # df.loc[df, 'daily_vehicles'](60.0/df_headways[hdwy_cols])
# df_headways.index = df_headways.LineID
# _df = (60.0/df_headways[hdwy_cols])
# _df = _df.replace(np.inf, 0)
# _df = _df.sum(axis=1)
# _df = pd.DataFrame(_df, columns=['daily_vehicles'])
# # _df = _df.reset_index()
# df = df.merge(_df, left_on='LineID', right_index=True)

# df_transit = df_transit.merge(df[['LineID','daily_vehicles','ij']], left_on='id', right_on='ij')

# df

# def get_transit_miles(geo_df, fname, year):
#     geo_df['new_route_type'] = geo_df['route_type'].map(route_type_map)
#     _df = pd.read_csv(os.path.join(input_dir,fname))

#     geo_df = geo_df.to_crs(crs)
#     geo_df['miles'] = geo_df.length/5280.0
#     df = geo_df.groupby('new_route_type').sum()[['miles']].T
#     df = df*annual_factor
#     for col in ['DRRevMi','VPRevMi','MBRevMi','RBRevMi','MGRevMi','SRRevMi','HRRevMi','CRRevMi']:
#         if col not in df.columns:
#             df[col] = 0
#     df['Year'] = year
#     df['Geo'] = regional_geo

#     return df

# df_18 = get_transit_miles(geo_df, fname, '2018')
# df_50 = get_transit_miles(geo_df, fname, '2050')
# df = df_18.append(df_50)
# df.to_csv(os.path.join(output_dir,fname), index=False)



In [113]:
# # df_transit.groupby('route_type').sum()[['miles']].T
# df_length = df.groupby('LineID').sum()[['length']]
# df = df_length.merge(_df, left_index=True, right_index=True)
# df['revenue_miles'] = df['length']*df['daily_vehicles']
# df['revenue_miles'].sum()*annual_factor

40616952.37306619

In [122]:
# # Calculate total daily transit vehicles
# # df_headways
# hdwy_cols = df_headways.columns.drop(['Unnamed: 0','LineID','id'])
# # df.loc[df, 'daily_vehicles'](60.0/df_headways[hdwy_cols])
# df_headways.index = df_headways.LineID
# _df = (60.0/df_headways[hdwy_cols])
# _df = _df.replace(np.inf, 0)
# _df = _df.sum(axis=1)

# _df = pd.DataFrame(_df, columns=['daily_vehicles'])
# _df.reset_index()

# Region

In [ ]:
########### Optional ##########

'other_ops_effectiveness.csv'
# This file is used to specify the delay effects of operations in different road classes and is optional (only needed if user wants to modify the values). The file is used in the Initialize (VETravelPerformance) and CalculateRoadPerformance modules.

'region_ave_fuel_carbon_intensity.csv'
#This file is used to specify the average carbon density for different vehicle types and is optional (only needed if user wants to modify the values). The file is used in the Initialize (VETravelPerformance) and CalculateCarbonIntensity modules.

'region_base_year_hvytrk_dvmt.csv'
# This file is used to specify the heavy truck dvmt for base year and is optional. The file is used in the Initialize (VETravelPerformance), CalculateBaseRoadDvmt and CalculateFutureRoadDvmt modules.

'region_carsvc_powertrain_prop.csv'
# This file is used to specify the powertrain proportion of car services and is optional. The file is used in the Initialize (VETravelPerformance), AssignHhVehiclePowertrain and AdjustHhVehicleMpgMpkwh modules.

'region_comsvc_powertrain_prop.csv'
#This file is used to specify the powertrain proportion of commercial vehicles and is optional. The file is used in the Initialize (VEPowertrainsAndFuels) and CalculateComEnergyAndEmissions modules.

'region_hvytrk_powertrain_prop.csv'
#This file is used to specify the powertrain proportion of heavy duty trucks and is optional. The file is used in the Initialize (VEPowertrainsAndFuels) and CalculateComEnergyAndEmissions modules.

for fname in ['other_ops_effectiveness.csv',
            'region_ave_fuel_carbon_intensity.csv',
            'region_base_year_hvytrk_dvmt.csv',
            'region_carsvc_powertrain_prop.csv',
            'region_comsvc_powertrain_prop.csv',
            'region_hvytrk_powertrain_prop.csv']:
    try:
        df = pd.read_csv(os.path.join(input_dir,fname))
    except:
        print(fname + ': file not available')
        next
        # print(fname)
    df['Geo'] = regional_geo
    if 'Year' in df.columns:
        df['Year'] = df['Year'].replace({2010: 2018, 2038: 2050})
    df.to_csv(os.path.join(output_dir,fname), index=False)


#### The following regional levels inputs are required


In [ ]:
fname = 'region_comsvc_lttrk_prop.csv'
# This file supplies data for the light truck proportion of commercial vehicles and is used in the CalculateComEnergyAndEmissions module.

########## FIXME: assuming 50% for now?


# ComSvcLtTrkProp: Regional proportion of commercial service vehicles that are light trucks Here is a snapshot of the file:

df = pd.DataFrame([0.5]).T
df.columns = ['ComSvcLtTrkProp']
df['Year'] = '2018'
df['Geo'] = regional_geo


df50 = pd.DataFrame([0.5]).T
df50.columns = ['ComSvcLtTrkProp']
df50['Year'] = '2050'
df50['Geo'] = regional_geo

df = df.append(df50)
df.to_csv(os.path.join(output_dir,fname), index=False)

In [ ]:
fname = 'region_hh_driver_adjust_prop.csv'
#This file specifies the relative driver licensing rate relative to the model estimation data year and is used in the AssignDrivers module.

## FIXME: set to 1

# Drv15to19AdjProp: Target proportion of unadjusted model number of drivers 15 to 19 years old (1 = no adjustment)
# Drv20to29AdjProp: Target proportion of unadjusted model number of drivers 20 to 29 years old (1 = no adjustment)
# Drv30to54AdjProp: Target proportion of unadjusted model number of drivers 30 to 54 years old (1 = no adjustment)
# Drv55to64AdjProp: Target proportion of unadjusted model number of drivers 55 to 64 years old (1 = no adjustment)
# Drv65PlusAdjProp: Target proportion of unadjusted model number of drivers 65 or older (1 = no adjustment)


df = pd.DataFrame([1,1,1,1,1]).T
df.columns = ['Drv15to19AdjProp','Drv20to29AdjProp','Drv30to54AdjProp','Drv55to64AdjProp','Drv65PlusAdjProp']
df['Year'] = '2018'
df['Geo'] = regional_geo
# df

df50 = pd.DataFrame([1,1,1,1,1]).T
df50.columns = ['Drv15to19AdjProp','Drv20to29AdjProp','Drv30to54AdjProp','Drv55to64AdjProp','Drv65PlusAdjProp']
df50['Year'] = '2050'
df50['Geo'] = regional_geo
df_50
df = df.append(df50)
df.to_csv(os.path.join(output_dir,fname), index=False)

In [ ]:

fname = 'region_prop_externalities_paid.csv'
#This file supplies data for climate change and other social costs and is used in the CalculateVehicleOperatingCost module.

# PropClimateCostPaid: Proportion of climate change costs paid by users (i.e. ratio of carbon taxes to climate change costs
# PropOtherExtCostPaid: Proportion of other social costs paid by users

df = pd.DataFrame([0,0]).T
df.columns = ['PropClimateCostPaid','PropOtherExtCostPaid']
df['Year'] = '2018'
df['Geo'] = regional_geo
# df

df50 = pd.DataFrame([0,0]).T
df50.columns = ['PropClimateCostPaid','PropOtherExtCostPaid']
df50['Year'] = '2050'
df50['Geo'] = regional_geo
df_50
df = df.append(df50)
df.to_csv(os.path.join(output_dir,fname), index=False)

### Compare Outputs

In [ ]:
import filecmp
import glob
# print()
files_target = glob.glob(r"C:\Workspace\VisionEval\models\VERSPM\inputs\*")
files_new = glob.glob(r"C:\Workspace\VisionEval\input_creation\psrc_inputs\*")
# match, mismatch, errors = filecmp.cmpfiles('C:\Workspace\VisionEval\input_creation\psrc_inputs', 'C:\Workspace\VisionEval\models\VERSPM\inputs', files)


# Get rap file names only
files_target = [i.split('\\')[-1] for i in files_target]
files_new = [i.split('\\')[-1] for i in files_new]


[i for i in files_target if i not in files_new]



In [ ]:
# Load the remaining missing files
# Keeping default values from RVMPO for now

fname = 'azone_hh_ave_veh_per_driver.csv'
df = pd.read_csv(os.path.join(input_dir,fname))
df['Year'] = ['2018','2050']
df['Geo'] = regional_geo
df.to_csv(os.path.join(output_dir,fname), index=False)

In [ ]:
fname = 'azone_hh_lttrk_prop.csv'
df = pd.read_csv(os.path.join(input_dir,fname))
df['Year'] = ['2018','2050']
df['Geo'] = regional_geo
df.to_csv(os.path.join(output_dir,fname), index=False)

In [ ]:
fname = 'region_base_year_dvmt.csv'
df = pd.read_csv(os.path.join(input_dir,fname))
# df['Year'] = ['2018','2050']
# df['Geo'] = regional_geo
df.to_csv(os.path.join(output_dir,fname), index=False)


In [ ]:
fname = 'region_co2e_costs.csv'
df = pd.read_csv(os.path.join(input_dir,fname))
df['Year'] = ['2018','2050']
# df['Geo'] = regional_geo
df.to_csv(os.path.join(output_dir,fname), index=False)

In [ ]:
fname = 'region_comsvc_ave_veh_age.csv'
df = pd.read_csv(os.path.join(input_dir,fname))
df['Year'] = ['2018','2050']
# df['Geo'] = regional_geo
df.to_csv(os.path.join(output_dir,fname), index=False)

In [ ]:
fname = 'region_comsvc_veh_mean_age.csv'
df = pd.read_csv(os.path.join(input_dir,fname))
df['Year'] = ['2018','2050']
# df['Geo'] = regional_geo
df.to_csv(os.path.join(output_dir,fname), index=False)

In [ ]:
fname = 'region_hh_ave_driver_per_capita.csv'
df = pd.read_csv(os.path.join(input_dir,fname))
df['Year'] = ['2018','2050']
# df['Geo'] = regional_geo
df.to_csv(os.path.join(output_dir,fname), index=False)

In [ ]:
fname = 'region_road_cost.csv'
df = pd.read_csv(os.path.join(input_dir,fname))
df['Year'] = ['2018','2050']
# df['Geo'] = regional_geo
df.to_csv(os.path.join(output_dir,fname), index=False)

In [147]:
gdf_shp = read_from_sde(connection_string, 'blockgrp2020', version, crs=crs, is_table=False)

C:\Users\Modeller\Anaconda3\envs\summary\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [148]:
gdf_shp.to_file(r'C:\Workspace\VisionEval\models\VERSPM\blockgrp2020.shp')

In [150]:
# gdf_shp